In [1]:
import google.generativeai as genai
#import env
import os
from dotenv import load_dotenv
load_dotenv()

GCP_KEY = os.getenv("GCP_KEY")


genai.configure(api_key=GCP_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

c:\Users\chavv\anaconda\envs\gemini\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import base64

# Function to read and base64 encode the PDF
def encode_pdf(file_path):
    with open(file_path, "rb") as pdf_file:
        pdf_data = pdf_file.read()
    return base64.standard_b64encode(pdf_data).decode("utf-8")

# Replace with your local PDF file paths
pdf_files = [
    "D:/github/OCR/Invoice-635286.pdf",
    "D:/github/OCR/Invoice-640419.pdf",
    "D:/github/OCR/wordpress-pdf-invoice-plugin-sample.pdf"
]

# Summarize each PDF
for pdf_file in pdf_files:
    encoded_pdf = encode_pdf(pdf_file)
    prompt = "Summarize this document"

    response = model.generate_content(
        [{'mime_type': 'application/pdf', 'data': encoded_pdf},
        prompt]
    )
    
    print(f"Summary for {pdf_file}:")
    print(response.text)
    print("=" * 80)

Summary for D:/github/OCR/Invoice-635286.pdf:
This is an invoice from IT Works to ACME Systems Inc. for concierge services.  The invoice number is 635286 and the date is 2017-12-06.  One concierge service was provided at a cost of 226351 EUR, resulting in a subtotal of 188626 EUR, a tax of 37725.2 EUR, and a total of 226351 EUR.  The invoice is payable within 20 days of the issue date.  ACME Systems Inc.'s address is Somewhere Road 59, Bucharest, Romania.  IT Works' address is Via Prossima 12, Torino, Italy.
Summary for D:/github/OCR/Invoice-640419.pdf:
This is an invoice from Clipboard Papers to ACME Systems Inc. for concierge services. The invoice number is 640419, and the date is June 18, 2017.  One concierge service was provided at a cost of 187,124 RON.  Including tax (37,424.8 RON), the total invoice amount is 224,549 RON.  Payment is due seven days after receipt of the invoice.
Summary for D:/github/OCR/wordpress-pdf-invoice-plugin-sample.pdf:
This is an invoice from Sliced Invo